In [1]:
from web3 import Web3
from eth_account import Account
from eth_account.messages import encode_defunct
from eth_typing import URI
from typing import List

In [2]:
import uniswap
import uniswap.util as uu

In [3]:
# info: https://gov.uniswap.org/t/deploy-uniswap-v3-on-filecoin-virtual-machine-fvm/21309/19
uniswap_obj = uniswap.Uniswap(
    "",  # Put metamask public wallet address here
    "",  # put metamask wallet private key here
    "https://api.node.glif.io/rpc/v1",
    version=3,
)

In [4]:
# setup stuff for filecoin specific
uniswap_obj.factory_contract = uu._load_contract(
    uniswap_obj.w3, abi_name="uniswap-v3/factory", 
    address="0xB4C47eD546Fc31E26470a186eC2C5F19eF09BA41",  # this is the correct thing, from Yee Han
)
uniswap_obj.quoter_addr = uniswap._str_to_addr("0xE45C06922228A33fFf1ED54638A0db78f69F9780")
uniswap_obj.quoter = uu._load_contract(
    uniswap_obj.w3, abi_name="uniswap-v3/quoter", address=uniswap_obj.quoter_addr
)

uniswap_obj.router_address = uniswap._str_to_addr(
    "0x83702C6356A1028A900F83d446D189a31646a16b"
)
uniswap_obj.router = uu._load_contract(
    uniswap_obj.w3, abi_name="uniswap-v3/router", address=uniswap_obj.router_address
)
uniswap_obj.positionManager_addr = uniswap._str_to_addr(
    "0x4cd986dD509fbB6A695aE971d5C56c8795f640ee"
)
uniswap_obj.nonFungiblePositionManager = uu._load_contract(
    uniswap_obj.w3,
    abi_name="uniswap-v3/nonFungiblePositionManager",
    address=uniswap_obj.positionManager_addr,
)
multicall2_addr = uniswap._str_to_addr(
    "0xffd927d6F17495B28635DD49d24638e97BD8c8b8"
)
uniswap_obj.multicall2 = uu._load_contract(
    uniswap_obj.w3, abi_name="uniswap-v3/multicall", address=multicall2_addr
)

In [5]:
wpFIL_addr = "0x57E3BB9F790185Cfe70Cc2C15Ed5d6B84dCf4aDb"
wFIL_addr = "0x60E1773636CF5E4A227d9AC24F20fEca034ee25A"

In [6]:
uniswap_obj.get_token_balance(wFIL_addr)

150000000000000000

In [7]:
int(.15*(10**18))

150000000000000000

In [8]:
uniswap_obj.get_token_balance(wpFIL_addr)

150172545065716058

In [9]:
if not uniswap_obj._is_approved(wpFIL_addr):
    uniswap_obj.approve(wpFIL_addr)

In [10]:
if not uniswap_obj._is_approved(wFIL_addr):
    uniswap_obj.approve(wFIL_addr)

In [11]:
tx_params = uniswap_obj._get_tx_params()
tx_params['gas'] = 150000000
tx_params['maxFeePerGas'] = 200000
tx_params['maxPriorityFeePerGas'] = 200000
print(tx_params)

{'from': '0x87c2067F301Bb48f377328cdda689578376b9C7d', 'value': 0, 'nonce': 26, 'gas': 150000000, 'maxFeePerGas': 200000, 'maxPriorityFeePerGas': 200000}


In [12]:
uniswap_obj._build_and_send_tx(
    uniswap_obj.router.functions.exactInputSingle(
        {
            "tokenIn": wFIL_addr,
            "tokenOut": wpFIL_addr,
            "fee": 3000,
            "recipient": uniswap_obj.address,
            "deadline": uniswap_obj._deadline(),
            "amountIn": int(.05*(10**18)),
            "amountOutMinimum": int(0),
            "sqrtPriceLimitX96": 0,
        }
    ),
    tx_params,
)

HexBytes('0x07c85d437c19864446932c094b1616a01381da27490f76cc74cdfd4dcd8af52f')